In [2]:
from dotenv import load_dotenv

load_dotenv()

from langchain_iris import IRISVector
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

/home/anish/experiments/oasis-scripts/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
username = 'SUPERUSER'
password = 'oasis' # Replace password with password you set 
# http://localhost:52773/csp/sys/UtilHome.csp
# change from 'SYS'
hostname = 'localhost' 
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
COLLECTION_NAME = "vectordb"

db = IRISVector(
    embedding_function=embeddings,
    dimension=768,
    collection_name=COLLECTION_NAME,
    connection_string=CONNECTION_STRING,
)

In [4]:
# db.delete_collection()

In [5]:
query = "tell me about breaches of customer privacy and losses"
docs_with_score = db.similarity_search_with_score(query, k=10)

In [6]:
for doc, score in docs_with_score[::-1]:
    # print("-" * 80)
    print("Score: ", score)
    print(doc.metadata)
    # print("-" * 80)

Score:  0.404625565802928
{'value': '0', 'metric': True, 'topic': 'G', 'description': 'Number of complaints (breaches of customer privacy) received from outside parties and substantiated by Qualcomm.', 'tags': ['privacy', 'cybersecurity'], 'id': '2022.204', 'year': '2022', 'company': 'Qualcomm Incorporated'}
Score:  0.404621352624312
{'value': 'Transparency Report', 'metric': False, 'topic': 'G', 'description': "Report providing information on Apple's transparency and compliance with government requests for customer data.", 'tags': ['transparency', 'data privacy'], 'id': '2022.71', 'year': '2022', 'company': 'Apple Inc.'}
Score:  0.404121006264615
{'value': '0', 'metric': True, 'topic': 'G', 'description': 'Number of complaints received from outside parties and substantiated by Qualcomm regarding breaches of customer privacy.', 'tags': ['customer privacy', 'compliance'], 'id': '2021.234', 'year': '2021', 'company': 'Qualcomm Incorporated'}
Score:  0.403576991734565
{'value': 'none', 'm

In [13]:
retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={
    "k": 50,
    "score_threshold": .15,
    "filter": {}
})
print(retriever)

tags=['IRISVector'] vectorstore=<langchain_iris.vectorstores.IRISVector object at 0x7f000b3cc040> search_type='similarity_score_threshold' search_kwargs={'k': 50, 'score_threshold': 0.15, 'filter': {}}


In [14]:
retriever.get_relevant_documents(
    query, consider_metadata=True
)

[Document(page_content='Substantiated complaints concerning breaches of customer privacy and losses of customer data.', metadata={'value': '2016418-1', 'metric': False, 'topic': 'S', 'description': 'Substantiated complaints concerning breaches of customer privacy and losses of customer data.', 'tags': ['customer privacy', 'compliance'], 'id': '2021.538', 'year': '2021', 'company': 'NVIDIA Corporation'}),
 Document(page_content='Substantiated complaints concerning breaches of customer privacy and losses of customer data.', metadata={'value': '418-1', 'metric': False, 'topic': 'G', 'description': 'Substantiated complaints concerning breaches of customer privacy and losses of customer data.', 'tags': ['customer privacy', 'data security'], 'id': '2020.561', 'year': '2020', 'company': 'NVIDIA Corporation'}),
 Document(page_content='Description of substantiated complaints concerning breaches of customer privacy and losses of customer data in 2020 and the three years prior.', metadata={'value

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_together import Together

# Initialize the Together model
model = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.7,
    max_tokens=1024,
    top_k=50,
)

# Extended template to include conversation history

company = "NVIDIA"

template = """
<s>[INST] You are an agent speaking with a representative from {company}.
Your goal is to assist the representative with questions regarding their sustainability practices.
You are provided context regarding {company}'s sustainablity practices and your goal is to compare
their practices with practices of other companies, for which context is given to you, in order to
help them improve.
Be critical with your response as well by noticing when metrics are missing or where they could be better.
Answer the question with a detailed response based only on the following context, incorporating data from other companies:
{context}

Conversation History:
{history}

Representative: {question} [/INST] 
"""
prompt = ChatPromptTemplate.from_template(template)

# Initialize conversation history
conversation_history = ""

def update_history(question, answer):
    global conversation_history
    conversation_history += f"Representative: {question}\nAgent: {answer}\n"

def get_response(input_query):
    global conversation_history
    # Define the chain with updated conversation history in the context
    chain = (
        {
            "context": retriever, 
            "history": lambda x: conversation_history, 
            "company": lambda x: company, 
            "question": RunnablePassthrough()
        }
        | prompt
        | model
        | StrOutputParser()
    )
    
    output = chain.invoke(input_query)
    update_history(input_query, output)
    return output

# Example usage
input_query = "How do we currently incorporate renewable energy?"
output = get_response(input_query)
print(output)

NVIDIA Corporation has been making strides in incorporating renewable energy into its operations. In FY19, 48% of NVIDIA's total electricity consumption came from renewable sources, and this increased to 56% in FY21. Furthermore, NVIDIA has been sourcing renewable energy through power purchase agreements (PPAs) or long-term contracts. In FY21, the company had 5.8 gigawatts of renewable energy via PPAs or long-term contracts. NVIDIA is also evaluating the implementation of onsite solar photovoltaic generation at its locations.

When compared to other companies, NVIDIA is doing well in terms of renewable energy sourcing, but there is still room for improvement. Apple Inc., for instance, has achieved 100% renewable electricity for its facilities and supply chain. In addition, Apple has invested in various types of renewable energy projects, including solar, wind, biogas fuel cells, and low-impact hydro projects. Meanwhile, Intel Corporation has increased its use of green power from 71% to

In [10]:
output = get_response('Speak more about the room for improvement. In what areas can we improve?')
print(output)

Based on the provided context, here are some areas where NVIDIA could consider for improvement:

1. More Ambitious Renewable Energy Goals: While NVIDIA has made significant strides in renewable energy, they could consider setting more ambitious goals, such as aiming for 100% renewable energy like some other tech companies.

2. Carbon Capture and Storage Technologies: NVIDIA could explore investing in carbon capture and storage technologies to reduce emissions. This could be particularly beneficial given their significant energy usage in data centers.

3. Supplier Performance: NVIDIA already requires their key manufacturing suppliers to report their energy usage, GHG emissions data, and reduction goals. However, they could further improve by setting clear expectations and standards for their suppliers' renewable energy usage and emissions reduction.

4. Employee Training: While NVIDIA fosters an environment of transparency and openness, they could further improve by providing training t

In [12]:
output = get_response('Nice. Provide some metrics on diversity in the workforce at NVIDIA.')
print(output)

Sure, I'd be happy to provide some metrics on diversity in NVIDIA's workforce.

In 2019, 89% of NVIDIA's employees said that the company has created an environment where people of diverse backgrounds can succeed. This is a strong indication of a diverse and inclusive workplace.

In terms of gender diversity, NVIDIA has been increasing the percentage of global female hires in engineering roles. However, the specific figures for this metric are not provided in the context.

Similarly, NVIDIA has been focusing on hiring underrepresented groups, but the specific metrics for this are also not provided.

In 2019, NVIDIA participated in the Grace Hopper Celebration of Women in Computing conference, which is a significant event for women in technology. The number of NVIDIA employees who joined the conference is referred to as 'NVIDIANs', but the specific number is not provided.

In terms of employee satisfaction, 97% of employees in 2021, 96% in 2020, and 90% in 2019 were proud to tell others 

In [11]:
input_query_prompt = f"<s>[INST] {input_query} [/INST] "
print(model(input_query_prompt))

1. Solar Power: NVIDIA has installed solar panels on the roofs of several of its buildings, which generate a significant portion of the company's energy needs.

2. Wind Energy: NVIDIA has also invested in wind energy. The company has a wind turbine at its headquarters in Santa Clara, California, which generates electricity for the campus.

3. Energy Efficient Data Centers: NVIDIA designs its data centers to be as energy efficient as possible. The company uses advanced cooling systems and energy-efficient servers to reduce its data centers' overall energy consumption.

4. Green Manufacturing: NVIDIA has implemented green manufacturing processes in its factories. These processes reduce waste, conserve resources, and lower greenhouse gas emissions.

5. Renewable Energy Credits (RECs): NVIDIA purchases renewable energy credits (RECs) to offset its carbon footprint. RECs represent the environmental attributes of renewable energy generation and are used to support and grow the renewable ener